In [1]:
# Dependencies and Setup
# !pip install citipy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import re
import plotly.offline as py
import sqlite3
from pprint import pprint
from sqlalchemy import create_engine

In [2]:
# 1) NYC_Health_Ratings (full list)
# 2) NYC_Violations_by_Restaurant (grouped by restaurant/violation code)
# 3) NYC_Restaurant_Coordinates (restaurants and geo information)

In [3]:
health_df = pd.read_csv('data/NYC_Restaurants.csv') # this is table 1

In [4]:
#converting all the dates to datetime to make easy for processing
health_df["GRADE DATE"]=pd.to_datetime(health_df["GRADE DATE"])
health_df["INSPECTION DATE"]=pd.to_datetime(health_df["INSPECTION DATE"])
health_df["RECORD DATE"]=pd.to_datetime(health_df["RECORD DATE"])
health_df.head()
# health_df.to_sql

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,2019-06-11,Violations were cited in the following area(s).,08C,Pesticide use not in accordance with label or ...,Not Critical,6,A,2019-06-11,2019-06-21,Cycle Inspection / Re-inspection
1,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,2019-06-11,Violations were cited in the following area(s).,10B,Plumbing not properly installed or maintained;...,Not Critical,6,A,2019-06-11,2019-06-21,Cycle Inspection / Re-inspection
2,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,2019-06-11,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,6,A,2019-06-11,2019-06-21,Cycle Inspection / Re-inspection
3,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,2019-06-06,Violations were cited in the following area(s).,06C,Food not protected from potential source of co...,Critical,8,A,2019-06-06,2019-06-21,Cycle Inspection / Initial Inspection
4,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,2019-06-06,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,8,A,2019-06-06,2019-06-21,Cycle Inspection / Initial Inspection


In [5]:
# Violation counts by Restaurant name and ID
violaton_count_df = health_df.groupby(["CAMIS"]).agg({"DBA":"first","VIOLATION CODE":"count"})
# violaton_count_df.DBA = violaton_count_df.DBA.astype(int)
violaton_count_df = violaton_count_df.rename(columns={"VIOLATION CODE":"Count of Violations","DBA":"Restaurant Name"})
violaton_count_df.index.name = "Restaurant ID"
violaton_count_df.head()
# violaton_count_df.to_sql

,Restaurant Name,Count of Violations
Restaurant ID,,
30075445,MORRIS PARK BAKE SHOP,3
30191841,DJ REYNOLDS PUB AND RESTAURANT,2
40362715,THE COUNTRY CAFE,3
40362869,SHASHEMENE INT'L RESTAURANT,2
40364286,PLAZA BAGELS & DELI,2


In [6]:
# Violation counts by Borough and Restaurant
violation_by_boro = health_df.groupby(["BORO","DBA"]).agg({"VIOLATION CODE":"count"})
violation_by_boro = violation_by_boro.rename(columns={"VIOLATION CODE":"Count of Violations"})
violation_by_boro = violation_by_boro.reset_index()
violation_by_boro.head()
# violation_by_boro.to_sql

,BORO,DBA,Count of Violations
0,BRONX,3-J RESTAURANT AND PIZZA,2
1,BRONX,619 BAR & RESTAURANT EL SALVADORENO,3
2,BRONX,AL CHOLO WEST INDIAN BAKERY & HEALTH FOOD STORE,1
3,BRONX,ALBERT'S COFFEE SHOP,2
4,BRONX,ANTHONY'S AND SALS PIZZA EXPRESS,3


In [7]:
# Violation counts by cuisine description
violation_by_cuisine = health_df.groupby(["CUISINE DESCRIPTION"]).agg({"VIOLATION CODE":"count"})
violation_by_cuisine = violation_by_cuisine.rename(columns={"VIOLATION CODE":"Count of Violations"})
violation_by_cuisine = violation_by_cuisine.reset_index()
violation_by_cuisine.head(25)
# violation_by_cuisine.to_sql

,CUISINE DESCRIPTION,Count of Violations
0,Afghan,7
1,African,12
2,American,1407
3,Armenian,3
4,Asian,86
5,Australian,3
6,Bagels/Pretzels,46
7,Bakery,176
8,Bangladeshi,8
9,Barbecue,16


In [8]:
restaurant_geo_df = pd.read_csv('data/Restaurant_Geo_Final.csv')
restaurant_geo_df.head(10)
# restaurant_geo.to_sql

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,address_full,Address,cities,lat,long
0,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,6/11/2019,Violations were cited in the following area(s).,...,6,A,6/11/2019,6/21/2019,Cycle Inspection / Re-inspection,1007 MORRIS PARK AVE 10462.0,1007 MORRIS PARK AVE 10462.0,"1007 Morris Park Ave, The Bronx, NY 10462, USA",40.848435,-73.856034
1,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,6/11/2019,Violations were cited in the following area(s).,...,6,A,6/11/2019,6/21/2019,Cycle Inspection / Re-inspection,1007 MORRIS PARK AVE 10462.0,1007 MORRIS PARK AVE 10462.0,"1007 Morris Park Ave, The Bronx, NY 10462, USA",40.848435,-73.856034
2,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,6/11/2019,Violations were cited in the following area(s).,...,6,A,6/11/2019,6/21/2019,Cycle Inspection / Re-inspection,1007 MORRIS PARK AVE 10462.0,1007 MORRIS PARK AVE 10462.0,"1007 Morris Park Ave, The Bronx, NY 10462, USA",40.848435,-73.856034
3,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,6/6/2019,Violations were cited in the following area(s).,...,8,A,6/6/2019,6/21/2019,Cycle Inspection / Initial Inspection,351 WEST 57 STREET 10019.0,351 WEST 57 STREET 10019.0,"351 W 57th St, New York, NY 10019, USA",40.767777,-73.984908
4,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,6/6/2019,Violations were cited in the following area(s).,...,8,A,6/6/2019,6/21/2019,Cycle Inspection / Initial Inspection,351 WEST 57 STREET 10019.0,351 WEST 57 STREET 10019.0,"351 W 57th St, New York, NY 10019, USA",40.767777,-73.984908
5,40362715,THE COUNTRY CAFE,MANHATTAN,60,WALL STREET,10005.0,3474279132,Sandwiches/Salads/Mixed Buffet,6/7/2019,Violations were cited in the following area(s).,...,20,Z,6/7/2019,6/21/2019,Cycle Inspection / Re-inspection,60 WALL STREET 10005.0,60 WALL STREET 10005.0,Error,0.000000,0.000000
6,40362715,THE COUNTRY CAFE,MANHATTAN,60,WALL STREET,10005.0,3474279132,Sandwiches/Salads/Mixed Buffet,6/7/2019,Violations were cited in the following area(s).,...,20,Z,6/7/2019,6/21/2019,Cycle Inspection / Re-inspection,60 WALL STREET 10005.0,60 WALL STREET 10005.0,Error,0.000000,0.000000
7,40362715,THE COUNTRY CAFE,MANHATTAN,60,WALL STREET,10005.0,3474279132,Sandwiches/Salads/Mixed Buffet,6/7/2019,Violations were cited in the following area(s).,...,20,Z,6/7/2019,6/21/2019,Cycle Inspection / Re-inspection,60 WALL STREET 10005.0,60 WALL STREET 10005.0,Error,0.000000,0.000000
8,40362869,SHASHEMENE INT'L RESTAURANT,BROOKLYN,195,EAST 56 STREET,11203.0,3474300871,Caribbean,6/14/2019,Violations were cited in the following area(s).,...,13,A,6/14/2019,6/21/2019,Cycle Inspection / Initial Inspection,195 EAST 56 STREET 11203.0,195 EAST 56 STREET 11203.0,"195 E 56th St, Brooklyn, NY 11203, USA",40.652219,-73.924453
9,40362869,SHASHEMENE INT'L RESTAURANT,BROOKLYN,195,EAST 56 STREET,11203.0,3474300871,Caribbean,6/14/2019,Violations were cited in the following area(s).,...,13,A,6/14/2019,6/21/2019,Cycle Inspection / Initial Inspection,195 EAST 56 STREET 11203.0,195 EAST 56 STREET 11203.0,"195 E 56th St, Brooklyn, NY 11203, USA",40.652219,-73.924453


In [9]:
markers_df = pd.read_csv('data/Map_Marker_File.csv')
markers_df = markers_df.rename(columns={"VIOLATION CODE":"Count of Violations"})
markers_df.head(10)
# restaurant_geo.to_sql

,DBA,BORO,CUISINE DESCRIPTION,GRADE,address_full,lat,long,Count of Violations
0,'CESCA,MANHATTAN,Italian,A,166 W 75TH ST 10023.0,40.780320,-73.979927,2
1,1 OR 8,BROOKLYN,Japanese,A,66 SOUTH 2 STREET 11249.0,40.714253,-73.965448,3
2,101 KENNEDY FRIED CHICKEN,QUEENS,American,Z,10815 101ST AVE 11419.0,40.687596,-73.835026,3
3,1080 BREW,QUEENS,CafÃ©/Coffee/Tea,Z,1080 WYCKOFF AVE 11385.0,40.694332,-73.902807,1
4,110 KENNEDY FRIED CHICKEN,STATEN ISLAND,Chicken,A,110 VICTORY BLVD 10301.0,40.637415,-74.079978,2
5,120 BAY CAFE,STATEN ISLAND,American,A,120 BAY STREET 10301.0,40.639466,-74.075794,2
6,18 RESTAURANT,MANHATTAN,Jewish/Kosher,A,240 EAST 81 STREET 10028.0,40.774702,-73.954650,4
7,1803,MANHATTAN,American,A,78 READE ST 10007.0,40.715428,-74.007204,3
8,2 BROS PIZZA,MANHATTAN,Pizza,Z,113 E 125TH ST 10035.0,40.805003,-73.938056,4
9,2 BROS PIZZA/ CORVO COFFEE,MANHATTAN,Pizza,Z,542 9 AVENUE 10018.0,40.756985,-73.993448,6


In [10]:
#Connect to local database
database_path = "NYC_Health_Ratings.sqlite"
engine = create_engine(f'sqlite:///{database_path}')

In [11]:
# Connect to the demo database
# connection  = sqlite3.connect("NYC_Health_Ratings.sqlite.db")

In [12]:
# Get a cursor object
# cursor      = connection.cursor()
# Execute the DROP Table SQL statement

In [13]:
# dropTableStatement = "DROP TABLE Violations_by_Borough"
# dropTableStatement = "DROP TABLE Violations_by_Cuisine"

In [14]:
# cursor.execute(dropTableStatement)

In [15]:
#Check for tables
engine.table_names()

['NYC_Health_Ratings',
 'Restaurant_Geo_Info',
 'Violations_by_Borough',
 'Violations_by_Cuisine',
 'Violations_by_Restaurant']

In [16]:
# 1) NYC_Health_Ratings (full list)
# 2) NYC_Violations_by_Restaurant (grouped by restaurant/violation code)
# 3) NYC_Restaurant_Coordinates (restaurants and geo information)

#Use pandas to load csv converted dataframe into database
health_df.to_sql(name='NYC_Health_Ratings', con=engine, if_exists='replace', index=False)
violaton_count_df.to_sql(name='Violations_by_Restaurant', con=engine, if_exists='replace', index=False)
restaurant_geo_df.to_sql(name='Restaurant_Geo_Info', con=engine, if_exists='replace', index=False)
violation_by_boro.to_sql(name='Violations_by_Borough', con=engine, if_exists='replace', index=False)
violation_by_cuisine.to_sql(name='Violations_by_Cuisine', con=engine, if_exists='replace', index=False)
markers_df.to_sql(name='Restaurant_Markers', con=engine, if_exists='replace', index=False)
# health_df.to_sql -- this is table 1
# violaton_count_df.to_sql -- this is table 2
# restaurant_geo.to_sql -- this is table 3
# violation_by_boro.to_sql -- extra
# violation_by_cuisine.to_sql -- extra

In [17]:
engine.table_names()

['NYC_Health_Ratings',
 'Restaurant_Geo_Info',
 'Restaurant_Markers',
 'Violations_by_Borough',
 'Violations_by_Cuisine',
 'Violations_by_Restaurant']

In [18]:
# Data retrieval test
pd.read_sql_query('select * from NYC_Health_Ratings', con=engine).head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,2019-06-11 00:00:00.000000,Violations were cited in the following area(s).,08C,Pesticide use not in accordance with label or ...,Not Critical,6,A,2019-06-11 00:00:00.000000,2019-06-21 00:00:00.000000,Cycle Inspection / Re-inspection
1,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,2019-06-11 00:00:00.000000,Violations were cited in the following area(s).,10B,Plumbing not properly installed or maintained;...,Not Critical,6,A,2019-06-11 00:00:00.000000,2019-06-21 00:00:00.000000,Cycle Inspection / Re-inspection
2,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,2019-06-11 00:00:00.000000,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,6,A,2019-06-11 00:00:00.000000,2019-06-21 00:00:00.000000,Cycle Inspection / Re-inspection
3,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,2019-06-06 00:00:00.000000,Violations were cited in the following area(s).,06C,Food not protected from potential source of co...,Critical,8,A,2019-06-06 00:00:00.000000,2019-06-21 00:00:00.000000,Cycle Inspection / Initial Inspection
4,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,2019-06-06 00:00:00.000000,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,8,A,2019-06-06 00:00:00.000000,2019-06-21 00:00:00.000000,Cycle Inspection / Initial Inspection


In [19]:
# Data retrieval test
pd.read_sql_query('select * from Violations_by_Restaurant', con=engine).head()

,Restaurant Name,Count of Violations
0,MORRIS PARK BAKE SHOP,3
1,DJ REYNOLDS PUB AND RESTAURANT,2
2,THE COUNTRY CAFE,3
3,SHASHEMENE INT'L RESTAURANT,2
4,PLAZA BAGELS & DELI,2


In [20]:
# Data retrieval test
pd.read_sql_query('select * from Restaurant_Geo_Info', con=engine).head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,address_full,Address,cities,lat,long
0,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,6/11/2019,Violations were cited in the following area(s).,...,6,A,6/11/2019,6/21/2019,Cycle Inspection / Re-inspection,1007 MORRIS PARK AVE 10462.0,1007 MORRIS PARK AVE 10462.0,"1007 Morris Park Ave, The Bronx, NY 10462, USA",40.848435,-73.856034
1,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,6/11/2019,Violations were cited in the following area(s).,...,6,A,6/11/2019,6/21/2019,Cycle Inspection / Re-inspection,1007 MORRIS PARK AVE 10462.0,1007 MORRIS PARK AVE 10462.0,"1007 Morris Park Ave, The Bronx, NY 10462, USA",40.848435,-73.856034
2,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,6/11/2019,Violations were cited in the following area(s).,...,6,A,6/11/2019,6/21/2019,Cycle Inspection / Re-inspection,1007 MORRIS PARK AVE 10462.0,1007 MORRIS PARK AVE 10462.0,"1007 Morris Park Ave, The Bronx, NY 10462, USA",40.848435,-73.856034
3,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,6/6/2019,Violations were cited in the following area(s).,...,8,A,6/6/2019,6/21/2019,Cycle Inspection / Initial Inspection,351 WEST 57 STREET 10019.0,351 WEST 57 STREET 10019.0,"351 W 57th St, New York, NY 10019, USA",40.767777,-73.984908
4,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,6/6/2019,Violations were cited in the following area(s).,...,8,A,6/6/2019,6/21/2019,Cycle Inspection / Initial Inspection,351 WEST 57 STREET 10019.0,351 WEST 57 STREET 10019.0,"351 W 57th St, New York, NY 10019, USA",40.767777,-73.984908


In [21]:
# Data retrieval test
pd.read_sql_query('select * from Violations_by_Borough', con=engine).head()

,BORO,DBA,Count of Violations
0,BRONX,3-J RESTAURANT AND PIZZA,2
1,BRONX,619 BAR & RESTAURANT EL SALVADORENO,3
2,BRONX,AL CHOLO WEST INDIAN BAKERY & HEALTH FOOD STORE,1
3,BRONX,ALBERT'S COFFEE SHOP,2
4,BRONX,ANTHONY'S AND SALS PIZZA EXPRESS,3


In [22]:
# Data retrieval test
pd.read_sql_query('select * from Violations_by_Cuisine', con=engine).head(30)

,CUISINE DESCRIPTION,Count of Violations
0,Afghan,7
1,African,12
2,American,1407
3,Armenian,3
4,Asian,86
5,Australian,3
6,Bagels/Pretzels,46
7,Bakery,176
8,Bangladeshi,8
9,Barbecue,16


In [23]:
# Data retrieval test
pd.read_sql_query('select * from Restaurant_Markers', con=engine).head()

,DBA,BORO,CUISINE DESCRIPTION,GRADE,address_full,lat,long,Count of Violations
0,'CESCA,MANHATTAN,Italian,A,166 W 75TH ST 10023.0,40.780320,-73.979927,2
1,1 OR 8,BROOKLYN,Japanese,A,66 SOUTH 2 STREET 11249.0,40.714253,-73.965448,3
2,101 KENNEDY FRIED CHICKEN,QUEENS,American,Z,10815 101ST AVE 11419.0,40.687596,-73.835026,3
3,1080 BREW,QUEENS,CafÃ©/Coffee/Tea,Z,1080 WYCKOFF AVE 11385.0,40.694332,-73.902807,1
4,110 KENNEDY FRIED CHICKEN,STATEN ISLAND,Chicken,A,110 VICTORY BLVD 10301.0,40.637415,-74.079978,2


In [24]:
test = pd.read_sql_query('select * from Restaurant_Markers', con=engine).head()
test.columns

Index(['DBA', 'BORO', 'CUISINE DESCRIPTION', 'GRADE', 'address_full', 'lat',
       'long', 'Count of Violations'],
      dtype='object')